In [12]:
from IPython.display import display, clear_output
from ipywidgets import widgets, Button, Layout
import sympy as sym
import math
import time
import matplotlib.pyplot as plt
import pandas as pd
from prettytable import PrettyTable

p = None
A = None
B = None
Z = 1
c = 2
d = 3
#JacobCoord(x, y, z, p, ToYac): Эта функция конвертирует точки между аффинными и Якобиановыми координатами.
#Если ToYac равно 1, 
#функция конвертирует из аффинных в Якобиановы координаты, иначе - наоборот.
def JacobCoord(x, y, z, p, ToYac):
    if ToYac == 1:
        if x == 0 and y == 1:
            return 1, 1, 0
        return mod(x/z^c, p), mod(y/z^d, p), mod(1/z, p)
    else:
        if x == 1 and y == 1 and z == 0:
            return 0, 1, 0
        return mod(x/z^c, p), mod(y/z^d, p) ,mod(1, p)
    
#double(X, Y, Z, A, p): Эта функция удваивает точку в якобиановых координатах. Функция возвращает координаты удвоенной точки.
def double(X, Y, Z, A, p):
    T1 = mod(X^2, p)
    T2 = mod(Y^2, p)
    T3 = mod(T2^2, p)
    C = mod(Z^2, p)
    
    S = mod(2*((X + T2)^2 - T1 - T3), p)
    
    M = mod(3*T1 + A*C^2, p)
    F = mod(M^2 - 2*S, p)
    
    X2 = F
    Y2 = mod(M*(S - F) - 8*T3, p)
    Z2 = mod((Y + Z)^2 - T2 - C, p)

    return X2, Y2, Z2

#addJacob(X1, Y1, Z1, X2, Y2, Z2, p): Эта функция добавляет 
#две точки в смешанных Якобианово-афинных координатах. Возвращает координаты результата.
def addJacob(X1, Y1, Z1, X2, Y2, Z2, p):
    if X2 == 0 and Y2 == 1:
        return X1, Y1, Z1
    
    T1 = mod(Z1^2, p)
    T2 = mod(Z2^2, p)
    U1 = mod(X1*T2, p)
    U2 = mod(X2*T1, p)
    
    S1 = mod(Y1*Z2*T2, p)
    S2 = mod(Y2*Z1*T1, p)
    H = mod(U2 - U1, p)
    I = mod((2*H)^2, p)
    
    J = mod(H*I, p)
    r = mod(2*(S2 - S1), p)
    V = mod(U1*I, p)
    
    X3 = mod(r^2 - J - 2*V, p)
    Y3 = mod(r*(V - X3) - 2*S1*J, p)
    Z3 = mod(((Z1 + Z2)^2 - T1 - T2)*H, p)

    return (X3, Y3, Z3)
#addPoints(P, Q, E): Эта функция принимает две точки P и Q в аффинных координатах, преобразует их в якобиановы координаты,
#добавляет их вместе с использованием функции addJacob, а затем преобразует результат обратно в аффинные координаты.
def addPoints(P, Q, E):
    z = randint(1, p-1)

    if P[0] == 0 and P[1] == 1:
        Xp, Yp, Zp = 1, 1, 0
    else:
        Xp, Yp, Zp = JacobCoord(P[0], P[1], z, p, 1)

    Xr, Yr, Zr = addJacob(Xp, Yp, Zp, Q[0], Q[1], 1, p)
    Xa, Ya, Za = JacobCoord(Xr, Yr, Zr, p, 0)

    return E(Xa, Ya, Za)

#doublePoint(P, E): Эта функция принимает точку P в аффинных координатах, преобразует ее в якобиановы координаты,
#удваивает ее с использованием функции double, а затем преобразует результат обратно в аффинные координаты.
def doublePoint(P, E):
    z = randint(1, p-1)
    
    Xp, Yp, Zp = JacobCoord(P[0], P[1], z, p, 1)
    Xd, Yd, Zd = double(Xp, Yp, Zp, A, p)
    Xa, Ya, Za = JacobCoord(Xd, Yd, Zd, p, 0)
    
    return E(Xa, Ya, Za)

#binaryMult(P, E, k): Эта функция выполняет двоичное
#умножение точки P на число k с использованием сложения и удвоения точек эллиптической кривой.
def binaryMult(P, E, k):
    Q = E(0, 1, 0)
    k_bin = k.binary()
    t = len(k_bin)
    i = t - 1
    while i >= 0 :
        if k_bin[i] == '1':
            Q = addPoints(P, Q, E)
        P = doublePoint(P, E)
        i -= 1
    return Q

def binaryAlg(P, E, k):
    Q = E(0, 1, 0)
    k_bin = k.binary()
    i = len(k_bin) - 1
    while i >= 0:
        if k_bin[i] == '1':
            Q = Q + P
        P = 2*P
        i -= 1
    return Q

def devide_k_on_windows(k, d, r):
    kbin = k.binary()
    t = len(kbin)
    list_fi = []
    tmp = ""
    windows = -1
    i = t-1
    w = 0
    zw = 0
    nzw = 0
    if t < d:
        d = t
    while i >= 0:
        if nzw == 1:
            if w <= d - r:
                bits_to_read = r
            else:
                bits_to_read = d - w
                
            if bits_to_read > i:
                bits_to_read = i + 1
                
            for x in range(bits_to_read):
                tmp += kbin[i]
                i -= 1
                
            tmp_int = int(tmp, 2)
            new_zw = ""
            old_nzw = ""
            w += bits_to_read
            
            if w < d and i >= 0:
                continue
                
            if tmp_int == 0:
                nzw = 0
                zw = 1
                w = len(tmp)
                windows += 1
                list_fi.append(tmp)
                continue
            
            for x in range(len(tmp)-1, -1, -1):
                if tmp[x] == '0':
                    new_zw += '0'
                if tmp[x] == '1':
                    break

            for j in range (0, x+1):
                list_fi[windows] += tmp[j]
                
            #print(list_fi)
            w = len(list_fi[windows])
            
            if len(new_zw) > 0:
                windows += 1
                nzw = 0
                zw = 1
                tmp = ""
                w = len(new_zw)
                list_fi.append(new_zw)
                
            if w == d and nzw == 1:
                zw = 0
                nzw = 0
                tmp = ""
                
            continue
            
        if zw == 0 and nzw == 0:
            windows += 1
            list_fi.append("")

        if kbin[i] == '1':
            if zw == 1:
                windows += 1
                list_fi.append("")
                tmp = ""
                zw = 0
            nzw = 1
            list_fi[windows] += kbin[i]
            w = len(list_fi[windows])
        elif kbin[i] == '0':
            if nzw == 1:
                #print(list_fi)
                windows += 1
                list_fi.append("")
                tmp = ""
                nzw = 0
            zw = 1
            list_fi[windows] += kbin[i]
            w = len(list_fi[windows])
        
        i -= 1
    
    list_rev = []
    for x in range(0, len(list_fi)):
        list_rev.append(list_fi[x][::-1])
    
    return list_rev

def VLNW_check(P, E, k, d, r):
    i = 1
    list_i = {}
    while i <= 2^d - 1:
        #list_i[i] = i*P
        list_i[i] = binaryMult(P, E, i)
        i += 2
    Q = E(0, 1, 0)
    start_time = time.time()
    list_fk = devide_k_on_windows(k, d, r)
    print("Time spent making windows: ", time.time() - start_time)
    i = len(list_fk) - 1
    while i >= 0:
        j = 1
        while j<=len(list_fk[i]):
            Q = doublePoint(Q, E)
            j+=1  
        #Q = doublePoint(Q, E)
        #Q = binaryMult(Q, E, 2^len(list_fk[i]))
        #Q = (2^len(list_fk[i]))*Q
        pos = int(list_fk[i], 2)
        if pos != 0:
            Q = addPoints(list_i[pos], Q, E)
            #Q = Q + list_i[pos]
        i -= 1
    return Q

def generate_y_on_elliptic_curve(E, x):
    y_sqr = x**3 + E.a4() * x + E.a6()  # Уравнение кривой для определения y^2
    if y_sqr.is_square():
        y = y_sqr.sqrt()
        return y
    else:
        return -5

def generate_random_point_on_elliptic_curve(E):
    while True:
        x = Integer(randrange(1, p))  # Случайное x в диапазоне [1, p-1]
        y_sqr = x**3 + E.a4() * x + E.a6()  # Уравнение кривой для определения y^2
        if y_sqr.is_square():
            y = y_sqr.sqrt()
            return E(x, y)
def generate_random_point_on_elliptic_curve2(E):
    while True:
        x = Integer(randrange(1, p_input.value))  # Случайное x в диапазоне [1, p-1]
        y_sqr = x**3 + E.a4() * x + E.a6()  # Уравнение кривой для определения y^2
        if y_sqr.is_square():
            y = y_sqr.sqrt()
            return E(x, y)

def run_lab_variant(b):
    global p, A, B
    p = 115792089237316193816632940749697632408932724669461758699102671622853342616127
    A = 10001090682862736796505780343107022911835438229291751861888571869223050424594
    B = 45264756867680555803214833811970559410867867042681754140959938453766481155105
    Xg = 34131522064967883044818697086599304935071739419701303653954060188761196377435
    Yg = 22161743160862324650134218095423264926581638655388631138889754360346138331114
    k = 404040404040404040404040404040

    #k = 1 << 2048 - 1

    E = EllipticCurve(GF(p), [A, B])
    P = E(Xg, Yg)

    print("k =", k)
    print("k (bit) =", len(k.binary()))
    print()
    print("p (bit) =", len(p.binary()))
    print()
    
    print("kP Sage:", k*P)
    print()

    start_time = time.time()
    print("kP Binary algoritm (Sage):", binaryAlg(P, E, k))
    print("Time spent: ", time.time() - start_time)
    print()

    start_time = time.time()
    print("kP Binary algoritm:", binaryMult(P, E, k))
    print("Time spent: ", time.time() - start_time)
    print()

    start_time = time.time()
    print("kP VLNW algoritm:",VLNW_check(P, E, k, 4, 2))
    print("Time spent: ", time.time() - start_time)
    print()
    
def run_test(b):
    global p, A, B
    n_bits = [32, 64, 128, 256, 512, 1024, 2048]
    num_passes=10
    binary_times = []
    vlnw_4_2_times = []
    vlnw_5_2_times = []
    vlnw_6_2_times = []
    vlnw_4_3_times = []
    vlnw_5_3_times = []
    vlnw_6_3_times = []
    
    for n in n_bits:
        #print("n =", n)
        p = random_prime(2^n - 1, lbound=2^(n-1))
        #print("p =", p)
        print("p (bit) =", len(p.binary()))
        A, B = 0, 0
        E = None
        
        while not E:  # Генерируем кривую, пока не получим правильную
            A = randint(2, p - 2)
            B = randint(2, p - 2)
            try:
                E = EllipticCurve(GF(p), [A, B])
            except ValueError:
                E = None
        print("A = ", A)
        print("B = ", B)
        #print("E = ", E)

        P = generate_random_point_on_elliptic_curve(E)
        #print("P =", P)

#         k = Integer(randrange(2^(n-1), 2^(n-2)))
        k = random_prime(2^(n-1) - 1, lbound=2^(n-2))

        #print("k =", k)
        print("k (bit) =", len(k.binary()))
        # print()
    
        print("kP Sage:", k*P)
        print()
        
        start_time = time.time()
        result_binary = binaryMult(P, E, k)
        print("kP Binary algoritm:", result_binary)
        print("Time spent (Binary): ", time.time() - start_time)
        binary_times.append(time.time() - start_time)
        #print()

        start_time = time.time()
        result_VLNW_4_2 = VLNW_check(P, E, k, 4, 2)
        print("kP VLNW_4_2 algoritm:", result_VLNW_4_2)
        print("Time spent (VLNW_4_2): ", time.time() - start_time)
        vlnw_4_2_times.append(time.time() - start_time)
        print()
        
        start_time = time.time()
        result_VLNW_5_2 = VLNW_check(P, E, k, 5, 2)
        print("kP VLNW_5_2 algoritm:", result_VLNW_5_2)
        print("Time spent (VLNW_5_2): ", time.time() - start_time)
        vlnw_5_2_times.append(time.time() - start_time)
        print()
        
        start_time = time.time()
        result_VLNW_6_2 = VLNW_check(P, E, k, 6, 2)
        print("kP VLNW_6_2 algoritm:", result_VLNW_6_2)
        print("Time spent (VLNW_6_2): ", time.time() - start_time)
        vlnw_6_2_times.append(time.time() - start_time)
        print()
        
        start_time = time.time()
        result_VLNW_4_3 = VLNW_check(P, E, k, 4, 3)
        print("kP VLNW_4_3 algoritm:", result_VLNW_4_3)
        print("Time spent (VLNW_4_3): ", time.time() - start_time)
        vlnw_4_3_times.append(time.time() - start_time)
        print()
        
        start_time = time.time()
        result_VLNW_5_3 = VLNW_check(P, E, k, 5, 3)
        print("kP VLNW_5_3 algoritm:", result_VLNW_5_3)
        print("Time spent (VLNW_5_3): ", time.time() - start_time)
        vlnw_5_3_times.append(time.time() - start_time)
        print()
        
        start_time = time.time()
        result_VLNW_6_3 = VLNW_check(P, E, k, 6, 3)
        print("kP VLNW_6_3 algoritm:", result_VLNW_6_3)
        print("Time spent (VLNW_6_3): ", time.time() - start_time)
        vlnw_6_3_times.append(time.time() - start_time)
        print()
        
     # Create a DataFrame with results
    results = pd.DataFrame({
        'n_bits': n_bits,
        'Binary Time': binary_times,
        'VLNW(4,2) Time': vlnw_4_2_times,
        'VLNW(5,2) Time': vlnw_5_2_times,
        'VLNW(6,2) Time': vlnw_6_2_times,
        'VLNW(4,3) Time': vlnw_4_3_times,
        'VLNW(5,3) Time': vlnw_5_3_times,
        'VLNW(6,3) Time': vlnw_6_3_times
    })
    
    # Display the table
    print(results)
    # Создайте график
    plt.figure(figsize=(30, 15))

    # Отображаем данные на графике
    plt.plot(n_bits, binary_times, marker='o', label='Binary')
    plt.plot(n_bits, vlnw_4_2_times, marker='o', label='VLNW(4,2)')
    plt.plot(n_bits, vlnw_5_2_times, marker='o', label='VLNW(5,2)')
    plt.plot(n_bits, vlnw_6_2_times, marker='o', label='VLNW(6,2)')
    plt.plot(n_bits, vlnw_4_3_times, marker='o', label='VLNW(4,3)')
    plt.plot(n_bits, vlnw_5_3_times, marker='o', label='VLNW(5,3)')
    plt.plot(n_bits, vlnw_6_3_times, marker='o', label='VLNW(6,3)')

    # Добавляем заголовок и метки осей
    plt.title('Время выполнения программы', fontsize=24)
    plt.xlabel('Разрядность p', fontsize=18)
    plt.ylabel('Время (секунды)', fontsize=18)

    # Добавляем легенду с максимальным увеличением размера шрифта
    plt.legend(fontsize=18)

    # Настройка меток оси X
    plt.xticks(n_bits, fontsize=20)
    # Настройка меток оси Y
    plt.yticks(fontsize=20)

    # Отобразить график
    plt.grid()
    plt.show()
    
    # Создайте график
    plt.figure(figsize=(30, 15))

    # Отображаем данные на графике
#     plt.plot(n_bits, binary_times, marker='o', label='Binary')
    plt.plot(n_bits, vlnw_4_2_times, marker='o', label='VLNW(4,2)')
    plt.plot(n_bits, vlnw_5_2_times, marker='o', label='VLNW(5,2)')
    plt.plot(n_bits, vlnw_6_2_times, marker='o', label='VLNW(6,2)')
    plt.plot(n_bits, vlnw_4_3_times, marker='o', label='VLNW(4,3)')
    plt.plot(n_bits, vlnw_5_3_times, marker='o', label='VLNW(5,3)')
    plt.plot(n_bits, vlnw_6_3_times, marker='o', label='VLNW(6,3)')

    # Добавляем заголовок и метки осей
    plt.title('Время выполнения программы')
    plt.xlabel('Разрядность p')
    plt.ylabel('Время (секунды)')

    # Добавляем заголовок и метки осей
    plt.title('Время выполнения программы', fontsize=24)
    plt.xlabel('Разрядность p', fontsize=18)
    plt.ylabel('Время (секунды)', fontsize=18)

    # Добавляем легенду с максимальным увеличением размера шрифта
    plt.legend(fontsize=18)

    # Настройка меток оси X
    plt.xticks(n_bits, fontsize=20)
    # Настройка меток оси Y
    plt.yticks(fontsize=20)

    # Отобразить график
    plt.grid()
    plt.show()
    
    
def run_big_test(b):
    global p, A, B
    n_bits = [32, 64, 128, 256, 512, 1024, 2048]
    num_passes = 5

    binary_times = []
    vlnw_4_2_times = []
    vlnw_5_2_times = []
    vlnw_6_2_times = []
    vlnw_4_3_times = []
    vlnw_5_3_times = []
    vlnw_6_3_times = []
    
    for n in n_bits:
        print("p = ", n)
        # Initialize lists to store times for each pass
        binary_times_passes = []
        vlnw_4_2_times_passes = []
        vlnw_5_2_times_passes = []
        vlnw_6_2_times_passes = []
        vlnw_4_3_times_passes = []
        vlnw_5_3_times_passes = []
        vlnw_6_3_times_passes = []

        for zzz in range(num_passes):
            print("count = ", zzz)
            # Your existing code for a single pass here...
            p = random_prime(2^n - 1, lbound=2^(n-1))
            A, B = 0, 0
            E = None
            while not E:
                A = randint(2, p - 2)
                B = randint(2, p - 2)
                try:
                    E = EllipticCurve(GF(p), [A, B])
                except ValueError:
                    E = None
            P = generate_random_point_on_elliptic_curve(E)
            k = random_prime(2^(n-1) - 1, lbound=2^(n-2))

            start_time = time.time()
            result_binary = binaryMult(P, E, k)
            binary_times_passes.append(time.time() - start_time)

            start_time = time.time()
            result_VLNW_4_2 = VLNW_check(P, E, k, 4, 2)
            vlnw_4_2_times_passes.append(time.time() - start_time)

            start_time = time.time()
            result_VLNW_5_2 = VLNW_check(P, E, k, 5, 2)
            vlnw_5_2_times_passes.append(time.time() - start_time)

            start_time = time.time()
            result_VLNW_6_2 = VLNW_check(P, E, k, 6, 2)
            vlnw_6_2_times_passes.append(time.time() - start_time)

            start_time = time.time()
            result_VLNW_4_3 = VLNW_check(P, E, k, 4, 3)
            vlnw_4_3_times_passes.append(time.time() - start_time)

            start_time = time.time()
            result_VLNW_5_3 = VLNW_check(P, E, k, 5, 3)
            vlnw_5_3_times_passes.append(time.time() - start_time)

            start_time = time.time()
            result_VLNW_6_3 = VLNW_check(P, E, k, 6, 3)
            vlnw_6_3_times_passes.append(time.time() - start_time)

        # Calculate and append the average times for each algorithm
        binary_times.append(sum(binary_times_passes) / num_passes)
        vlnw_4_2_times.append(sum(vlnw_4_2_times_passes) / num_passes)
        vlnw_5_2_times.append(sum(vlnw_5_2_times_passes) / num_passes)
        vlnw_6_2_times.append(sum(vlnw_6_2_times_passes) / num_passes)
        vlnw_4_3_times.append(sum(vlnw_4_3_times_passes) / num_passes)
        vlnw_5_3_times.append(sum(vlnw_5_3_times_passes) / num_passes)
        vlnw_6_3_times.append(sum(vlnw_6_3_times_passes) / num_passes)

    # Create a DataFrame with results
    results = pd.DataFrame({
        'n_bits': n_bits,
        'Binary Time': binary_times,
        'VLNW(4,2) Time': vlnw_4_2_times,
        'VLNW(5,2) Time': vlnw_5_2_times,
        'VLNW(6,2) Time': vlnw_6_2_times,
        'VLNW(4,3) Time': vlnw_4_3_times,
        'VLNW(5,3) Time': vlnw_5_3_times,
        'VLNW(6,3) Time': vlnw_6_3_times
    })
    
    # Display the table
    print(results)
    
    # Создайте график
    plt.figure(figsize=(30, 15))

    # Отображаем данные на графике
    plt.plot(n_bits, binary_times, marker='o', label='Binary')
    plt.plot(n_bits, vlnw_4_2_times, marker='o', label='VLNW(4,2)')
    plt.plot(n_bits, vlnw_5_2_times, marker='o', label='VLNW(5,2)')
    plt.plot(n_bits, vlnw_6_2_times, marker='o', label='VLNW(6,2)')
    plt.plot(n_bits, vlnw_4_3_times, marker='o', label='VLNW(4,3)')
    plt.plot(n_bits, vlnw_5_3_times, marker='o', label='VLNW(5,3)')
    plt.plot(n_bits, vlnw_6_3_times, marker='o', label='VLNW(6,3)')

    # Добавляем заголовок и метки осей
    plt.title('Время выполнения программы', fontsize=24)
    plt.xlabel('Разрядность p', fontsize=18)
    plt.ylabel('Время (секунды)', fontsize=18)

    # Добавляем легенду с максимальным увеличением размера шрифта
    plt.legend(fontsize=18)

    # Настройка меток оси X
    plt.xticks(n_bits, fontsize=20)
    # Настройка меток оси Y
    plt.yticks(fontsize=20)

    # Отобразить график
    plt.grid()
    plt.show()
    
E1 = None
def generate_random_X1(b):
    #print("1", E1)
    random_X1 = generate_random_point_on_elliptic_curve2(E1)
    #print("random_X1 = ", random_X1)
    Xg_input.value = random_X1[0]
    
def generate_random_p(b):
    #print("p (bit) = ",p_bit_input )
    p = random_prime(2^p_bit_input.value - 1, lbound=2^(p_bit_input.value-1))
    p_input.value = p

    
def generate_random_AB(b):
    global E1
    A1, B1 = 0, 0
    E1 = None
    while not E1:
        A1 = randint(2, p_input.value - 2)
        B1 = randint(2, p_input.value - 2)
        try:
            E1 = EllipticCurve(GF(p_input.value), [A1, B1])
        except ValueError:
            E1 = None
    A_input.value = A1
    B_input.value = B1
    #print(E1)
    
    
def generate_random_k(b):
    #print("p (bit) = ",p_bit_input )
    k = random_prime(2^k_bit_input.value - 1, lbound=2^(k_bit_input.value-1))
    k_input.value = k
    
def generate_random_start(b):
    global p, A, B
    p = Integer(p_input.value)
    A = A_input.value
    B = B_input.value
    k = Integer(k_input.value)
    
    E = EllipticCurve(GF(p), [A, B])
#     Xg = 34131522064967883044818697086599304935071739419701303653954060188761196377435
#     Yg = 22161743160862324650134218095423264926581638655388631138889754360346138331114
    Xg = Xg_input.value
    Yg = generate_y_on_elliptic_curve(E, Xg)
    P = E(Xg, Yg)
    #k = bin(k)[2:]
    #p = bin(p)[2:]
    print("k =", k)
    print("k (bit) =", len(k.binary()))
    print()
    print("p =", p)
    print("p (bit) =", len(p.binary()))
    print()
    
    print("kP Sage:", k*P)
    print()

    start_time = time.time()
    print("kP Binary algoritm (Sage):", binaryAlg(P, E, k))
    print("Time spent: ", time.time() - start_time)
    print()

    start_time = time.time()
    print("kP Binary algoritm:", binaryMult(P, E, k))
    print("Time spent: ", time.time() - start_time)
    print()

    start_time = time.time()
    print("kP VLNW algoritm:",VLNW_check(P, E, k, 4, 2))
    print("Time spent: ", time.time() - start_time)
    print()
    
    
    
    
p_bit_input = widgets.IntText(description='Разр. p: ',layout=widgets.Layout(width="300px", height="50px"))
p_input = widgets.IntText(description='Введите p: ')
generate_button = widgets.Button(description='Сгенерировать p', layout=widgets.Layout(width="300px", height="50px"))
generate_button.on_click(generate_random_p)
  

A_input = widgets.IntText(description='A = ')
B_input = widgets.IntText(description='B = ')

generate_button_AB = widgets.Button(description='Сгенерировать A и B', layout=widgets.Layout(width="300px", height="50px"))
generate_button_AB.on_click(generate_random_AB)

Xg_input = widgets.IntText(description='X1 = ')

# Создаем кнопку для выполнения ввода
# input_button = widgets.Button(description="Ввести значения (p, A, B)", layout=widgets.Layout(width="300px", height="50px"))
# input_button.on_click(input_function)        

# input_button2 = widgets.Button(description="Ввести значение (X1)", layout=widgets.Layout(width="300px", height="50px"))
# input_button2.on_click(input_function2)

generate_button_X = widgets.Button(description='Сгенерировать X1', layout=widgets.Layout(width="300px", height="50px"))
generate_button_X.on_click(generate_random_X1)

generate_button_p = widgets.Button(description='Сгенерировать p', layout=widgets.Layout(width="300px", height="50px"))
generate_button_p.on_click(generate_random_p)

k_bit_input = widgets.IntText(description='Разр. k: ',layout=widgets.Layout(width="300px", height="50px"))
k_input = widgets.IntText(description='Введите k: ')
generate_button_k = widgets.Button(description='Сгенерировать k', layout=widgets.Layout(width="300px", height="50px"))
generate_button_k.on_click(generate_random_k)

lets_go = widgets.Button(description='Запустить алгоритмы', layout=widgets.Layout(width="300px", height="50px"))
lets_go.on_click(generate_random_start)
    

button_lab = widgets.Button(description="1. Запустить вариант из лабораторной", layout=widgets.Layout(width="500px", height="50px"))
button_test = widgets.Button(description="2. Запустить тест (t и p близки (32,64,128...))", layout=widgets.Layout(width="500px", height="50px"))
button_test_big = widgets.Button(description="3. Запустить тест (n прогонов 2 теста для каждого значения бит)", layout=widgets.Layout(width="500px", height="50px"))

button_lab.on_click(run_lab_variant)
button_test.on_click(run_test)
button_test_big.on_click(run_big_test)

display(button_lab, button_test,button_test_big,p_bit_input,p_input,generate_button_p, A_input,B_input,generate_button_AB,Xg_input,generate_button_X,k_bit_input,k_input,generate_button_k,lets_go)


Button(description='1. Запустить вариант из лабораторной', layout=Layout(height='50px', width='500px'), style=…

Button(description='2. Запустить тест (t и p близки (32,64,128...))', layout=Layout(height='50px', width='500p…

Button(description='3. Запустить тест (n прогонов 2 теста для каждого значения бит)', layout=Layout(height='50…

IntText(value=0, description='Разр. p: ', layout=Layout(height='50px', width='300px'))

IntText(value=0, description='Введите p: ')

...

Button(description='Сгенерировать X1', layout=Layout(height='50px', width='300px'), style=ButtonStyle())

IntText(value=0, description='Разр. k: ', layout=Layout(height='50px', width='300px'))

IntText(value=0, description='Введите k: ')

Button(description='Сгенерировать k', layout=Layout(height='50px', width='300px'), style=ButtonStyle())

Button(description='Запустить алгоритмы', layout=Layout(height='50px', width='300px'), style=ButtonStyle())